In [2]:
from pathlib import Path
from typing import Any
import json
import itertools as it

import altair as alt
from tools.schemapi.utils import SchemaInfo


def load_schema(fp: Path, /) -> dict[str, Any]:
    """Reads and returns the root schema from ``fp``."""
    with fp.open(encoding="utf8") as f:
        root_schema = json.load(f)
    return root_schema

def get_children(schema_info: SchemaInfo) -> set[str]:
    children: set[str] = set()

    for prop_info in it.chain(
        schema_info.properties.values(),
        schema_info.anyOf,
        [schema_info.child(schema_info.items)] if schema_info.items else [],
    ):
        if prop_info.refname and prop_info.is_reference():
            children.add(prop_info.refname)
        else:
            children.update(get_children(prop_info))

    return children

In [5]:
rootschema = load_schema(Path("../vega/altair/altair/vegalite/v5/schema/vega-lite-schema.json"))

In [6]:
definitions: dict[str, SchemaInfo] = {}
for name in rootschema["definitions"]:
    schema = SchemaInfo({"$ref": "#/definitions/" + name}, rootschema=rootschema)
    definitions[name] = schema

In [7]:
# Key is the name of a definition and the value is a unique list of names of definitions
# that are children of the key definition.
graph: dict[str, list[str]] = {}
for name, schema in definitions.items():
    graph[name] = list(get_children(schema))

In [13]:
with Path(f"docs/graphtypes_v{(alt.VEGALITE_VERSION).replace('.','_')}.json").open("w") as f:
    json.dump(graph, f, indent=2)

In [14]:
from pyvis.network import Network

net = Network(select_menu=True, filter_menu=True, directed=True)
nodes_to_exclude = set()

for node in graph:
    if node not in nodes_to_exclude:
        net.add_node(node, label=node, title=node)

for node, edges in graph.items():
    if node not in nodes_to_exclude:
        for edge in edges:
            if edge not in nodes_to_exclude:
                net.add_edge(node, edge)
net.show_buttons(filter_=['physics'])
net.save_graph(f"docs/graphtypes_v{(alt.VEGALITE_VERSION).replace('.','_')}.html")

In [ ]:
# I had to manualy move the created lib folder into the docs folder.